In [ ]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()
melb_df.drop(['index', 'Coordinates'], axis=1, inplace=True)
melb_df.head()

In [ ]:
total_rooms = melb_df['Rooms'] + melb_df['Bathroom'] + melb_df['Bedroom']
#display(total_rooms)
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
display(melb_df['MeanRoomsSquare'])

diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
display(melb_df['AreaRatio'])

In [ ]:
#display(melb_df['Date'])
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
#display(melb_df['Date'])
years_sold = melb_df['Date'].dt.year
print(years_sold)
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])
melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)
melb_df.head()

In [ ]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
display(delta_days)
display(delta_days.dt.days)

In [ ]:
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
display(melb_df['AgeBuilding'])
melb_df = melb_df.drop('YearBuilt', axis=1)
melb_df.head()

In [ ]:
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
melb_df.head()
weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)]['WeekdaySale'].count()
display(weekend_count)

In [ ]:
print(melb_df['Address'].nunique())
print(melb_df['Address'].loc[177])
print(melb_df['Address'].loc[1812])
print(melb_df['Address'].loc[9001])

In [ ]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

street_types = melb_df['Address'].apply(get_street_type)
display(street_types)

print(street_types.nunique())
display(street_types.value_counts())
popular_stypes = street_types.value_counts().nlargest(10).index
print(popular_stypes)
# Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
display(melb_df['StreetType'])

In [10]:
def get_weekend(weekday):
    if weekday in [0, 1, 2 , 3, 4]:
        return 0
    else:
        return 1
    
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend) 
#melb_df.tail() 

weekend_price = melb_df[melb_df['Weekend'] == 1]['Price'].mean()
display(round(weekend_price))

1081199

In [ ]:
popular_sellers = melb_df['SellerG'].value_counts().nlargest(49).index
display(popular_sellers)
melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_sellers else 'other')
#display(melb_df)
min_Nelson = melb_df[melb_df['SellerG'] == 'Nelson']['Price'].min()
min_other = melb_df[melb_df['SellerG'] == 'other']['Price'].min()
display(round(min_Nelson/min_other, 1))

In [ ]:
test_series_1 = pd.Series([
            'Опыт работы 8 лет 3 месяца',
            'Опыт работы 3 года 5 месяцев',
            'Опыт работы 1 год 9 месяцев',
            'Опыт работы 3 месяца',
            'Опыт работы 6 лет'
        ])

test_series_2 = pd.Series([
            'Опыт работы 5 лет',
            'Опыт работы 5 месяцев',
            'Опыт работы 1 год 1 месяц',
            'Опыт работы 3 месяца',
            'Опыт работы 7 лет'
        ])

def get_experience(arg):
    year = 0
    month = 0
    exclude_list_year = ['год', 'года', 'лет']
    exclude_list_month = ['месяц', 'месяца', 'месяцев']
    expirience_list = arg.split(' ')
    for i in range(len(expirience_list)):
        if expirience_list[i] in exclude_list_month:
            month = expirience_list[i-1]
        if expirience_list[i] in exclude_list_year:
            year = expirience_list[i-1]
    return int(month) + int(year)*12

test_series_1.apply(get_experience)  
test_series_2.apply(get_experience)         
        
    
    

In [ ]:
# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtypes) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
display(unique_counts)

In [ ]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())
print(melb_df['Regionname'].cat.categories)

In [ ]:
melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
display(melb_df['Type'])

melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

In [ ]:
display(melb_df.info())
popular_suburb = melb_df['Suburb'].value_counts().nlargest(119).index
melb_df['Suburb'] = melb_df['Suburb'].apply(lambda x: x if x in popular_suburb else 'other')
melb_df['Suburb'] = melb_df['Suburb'].astype('category')
display(melb_df.info())
